In [2]:
import tensorflow as tf

# Tensors

In [4]:
# from array
x = [[2.]]
m = tf.matmul(x, x)

m

<tf.Tensor: id=5, shape=(1, 1), dtype=float32, numpy=array([[4.]], dtype=float32)>

In [5]:
# constant tensor
a = tf.constant([[1, 2],
                 [3, 4]])

a

<tf.Tensor: id=7, shape=(2, 2), dtype=int32, numpy=
array([[1, 2],
       [3, 4]], dtype=int32)>

In [6]:
# operations
b = tf.add(a, 1)

b

<tf.Tensor: id=10, shape=(2, 2), dtype=int32, numpy=
array([[2, 3],
       [4, 5]], dtype=int32)>

In [8]:
# operators
c = a * b

c

<tf.Tensor: id=14, shape=(2, 2), dtype=int32, numpy=
array([[ 2,  6],
       [12, 20]], dtype=int32)>

In [11]:
# information
c.shape

TensorShape([2, 2])

In [12]:
# to numpy
c.numpy()

array([[ 2,  6],
       [12, 20]], dtype=int32)

In [13]:
# auto differentiation
w = tf.Variable([[1.0]])

with tf.GradientTape() as tape:
  loss = w * w

grad = tape.gradient(loss, w)

print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


# Training

In [14]:
(mnist_images, mnist_labels), _ = tf.keras.datasets.mnist.load_data()

dataset = tf.data.Dataset.from_tensor_slices(
  (tf.cast(mnist_images[...,tf.newaxis]/255, tf.float32),
   tf.cast(mnist_labels,tf.int64))).shuffle(1000).batch(32)

11493376/11490434 [==============================] - 8s 1us/step


In [15]:
mnist_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16,[3,3], activation='relu',
                         input_shape=(None, None, 1)),
  tf.keras.layers.Conv2D(16,[3,3], activation='relu'),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(10)
])

In [23]:
optimizer = tf.keras.optimizers.Adam()

loss_history = []
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [24]:
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = mnist_model(images, training=True)

        # Add asserts to check the shape of the output.
        tf.debugging.assert_equal(logits.shape, (32, 10))

        loss_value = loss_object(labels, logits)

    loss_history.append(loss_value.numpy().mean())
    grads = tape.gradient(loss_value, mnist_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, mnist_model.trainable_variables))

In [25]:
def train():
    for epoch in range(3):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(images, labels)
    print('Epoch {} finished'.format(epoch))

In [26]:
train()

Epoch 2 finished


In [29]:
loss_history[::200]

[2.304679,
 2.2282329,
 2.145129,
 1.8389103,
 1.9019797,
 1.8152134,
 1.7069305,
 1.7986327,
 1.497,
 1.3395107,
 1.7649226,
 1.3653595,
 1.6250725,
 1.6987095,
 1.5932025,
 1.6115596,
 1.7175161,
 1.340711,
 1.5979251,
 1.3701911,
 1.1830213,
 1.3882793,
 1.3152442,
 1.4229826,
 1.2129433,
 1.2245777,
 1.4503552,
 1.0775992,
 1.2430215]